# 2. Arthemetic Operations on Images

## Goal
* Learn several arithmetic operations on images like addition, subtraction, bitwise operations etc.
* You will learn these functions : cv2.add(), cv2.addWeighted() etc.


In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import cv2 as cv

## Image Addition

In [2]:
# pixel addition overflows with uint8 as the range of piexel is represented by 0-255
x = np.uint8([250])
y = np.uint8([10])

# modulus operation in Numpy
print(x+y) # 250+10 = 260 % 256 = 4

# satured operation in OpenCV
print(cv.add(x,y)) # 250+10 = 260 => 255

[4]


## Image Blending

This is also image addition but different weights are assigned to different images so that it gives a feeling of blending or transparency. Images are added as per the equation below:

$$g(x) = (1 - \alpha)f_0(x) + \alpha f_1(x)$$

By varying $\alpha$ from 0 $\rightarrow$ 1, you can perform a cool transition between one image to another.

Here I took two images to blend them together. First image is given a weight of 0.7 and second image is given 0.3. cv2.addWeighted() applies following equation on the image.

$$dst = \alpha \cdot img1 + \beta \cdot img2 + \gamma$$

Here $\gamma$ is taken as zero.



In [ ]:
img1 = cv.imread('messi5.jpg')
img2 = cv.imread('opencv-logo.png')

# add two images with the same size
dst = cv.addWeighted(img1, 0.7, img2, 0.3, 0)

## Bitwise Operations

This includes bitwise AND, OR, NOT and XOR operations. They will be highly useful while extracting any part of the image (as we will see in coming chapters), defining and working with non-rectangular ROI etc. Below we will see an example on how to change a particular region of an image.

I want to put OpenCV logo above an image. If I add two images, it will change color. If I blend it, I get an transparent effect. But I want it to be opaque. If it was a rectangular region, I could use ROI as we did in last chapter. But OpenCV logo is a not a rectangular shape. So you can do it with bitwise operations as below:

In [ ]:
# load two images
img1 = cv.imread('messi5.jpg')
img2 = cv.imread('opencv-logo.png')

# put logo on top-left corner, create a ROI
rows, cols, channels = img2.shape
roi = img1[0:rows, 0:cols]

# create a mask of logo and create its inverse mask
img2gray = cv.cvtColor(img2, cv.COLOR_BGR2GRAY)
ret, mask = cv.threshold(img2gray, 10, 255, cv.THRESH_BINARY)
mask_inv = cv.bitwise_not(mask)

# black-out the area of logo in ROI
img1_bg = cv.bitwise_and(roi, roi, mask=mask_inv)

# take only region of logo from logo image
img2_fg = cv.bitwise_and(img2, img2, mask=mask)

# put logo in ROI and modify the main image
dst = cv.add(img1_bg, img2_fg)
img1[0:rows, 0:cols] = dst

